In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm.notebook import tqdm

In [ ]:
# read in portfolio

df_all = pd.read_csv('/content/gdrive/My Drive/capstone/data for trend/Results_20100630_20200630.csv')
df = df_all[['INSTRID', 'LGCYINVESTORID', 'HOLDDATE', 'SHSHLDVAL']]
df['HOLDDATE'] = pd.to_datetime(df['HOLDDATE'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# # get an investor name dictionary
# investor_names = df[['LGCYINVESTORID','FULLNAME']].drop_duplicates()
# # get instrument name dictionary
# instrument_names = df_all[['INSTRID','COMNAME']].drop_duplicates()

In [ ]:
# investor_names.to_csv('investor_list.csv')

In [ ]:
# read in list of investors of interest

########
#NEED CHANGE LIST
########
df_investors = pd.read_csv('/content/gdrive/My Drive/capstone/full database/list_investors_225.csv')
# df_investors = df_investors[df_investors.isActive==True]

ls_investors = df_investors.LGCYINVESTORID.to_list()
print('There are {} investors which are we interested in.'.format(len(ls_investors)))

# get those in predefined list
df = df.loc[df['LGCYINVESTORID'].isin(ls_investors),['INSTRID', 'LGCYINVESTORID', 'HOLDDATE', 'SHSHLDVAL']]

# calculate the sum of SHSHLDVAL for each investor in each quarter
df ['SHSHLDVAL_Sum_ByQuarter'] = df.groupby(['LGCYINVESTORID','HOLDDATE']).transform('sum').drop(['INSTRID'], axis = 1)

# calculate the (13F only) pct of the SHSHLDVAL of each instrument in its investor's sum of SHSHLDVAL for each quarter 
# (i.e. revised version of 'PCTPORTFOLIO', so that sum up to 1)
df['PCTPORTFOLIO_13F'] = df['SHSHLDVAL'] / df['SHSHLDVAL_Sum_ByQuarter']

There are 225 investors which are we interested in.


## check

In [ ]:
## check whether any investor has missing 13F
for inv in investor_list:
  holddates = df[df['LGCYINVESTORID']==inv]['HOLDDATE'].drop_duplicates()
  earlist_date = min(holddates)
  latest_date = max(holddates)
  if (latest_date.year - earlist_date.year) * 4 + (latest_date.month - earlist_date.month) / 3 + 1 != len(holddates):
    print(inv)

5520904
3965008
5308615
4059340
2420555
6139212
2930530
2447262
2502059
2536411


In [ ]:
# check investor : 2502059, MKP Capital Management, L.L.C.
df[df['LGCYINVESTORID']==2502059]

,INSTRID,LGCYINVESTORID,HOLDDATE,SHSHLDVAL,SHSHLDVAL_Sum_ByQuarter,PCTPORTFOLIO_13F,PREVIOUS_PCTPORTFOLIO,delta
182713,8589940754,2502059,2017-06-30,1.520308e+06,1.129875e+07,0.134555,NaN,NaN
182813,8589940754,2502059,2019-12-31,5.484712e+07,7.309063e+08,0.075040,0.134555,0.557688
985627,8589968548,2502059,2016-12-31,2.959500e+06,9.951582e+07,0.029739,NaN,NaN
1206577,8589976864,2502059,2019-12-31,4.159933e+06,7.309063e+08,0.005691,NaN,NaN
1638108,8589996470,2502059,2017-03-31,4.886990e+08,4.958180e+08,0.985642,NaN,NaN
...,...,...,...,...,...,...,...,...
20445562,21581736200,2502059,2020-06-30,3.641508e+06,1.132120e+08,0.032165,NaN,NaN
20562825,21589564978,2502059,2018-09-30,6.532000e+04,3.308883e+08,0.000197,NaN,NaN
20562817,21589564978,2502059,2019-03-31,2.754013e+05,2.754013e+05,1.000000,0.000197,5065.650728
20747385,21604644667,2502059,2019-12-31,1.588410e+06,7.309063e+08,0.002173,NaN,NaN


In [ ]:
investor_names[investor_names['LGCYINVESTORID'] == 2502059]

,LGCYINVESTORID,FULLNAME
182713,2502059,"MKP Capital Management, L.L.C."


In [ ]:
instrument_names[instrument_names['INSTRID'] == 21589564978]

,INSTRID,COMNAME
20559357,21589564978,Hertz Global Holdings Ord Shs


##  Create Delta for portfolio

$$\delta = pctPortfolio_{t} -  pctPortfolio_{t-1}$$

investor i, instrument j, time T, data time range:2016-2019
1.  Consider $\delta$ at the first time point(2016Q2) as same as the pctPortfolio. Also, if the investor appeared in the market at 2018Q1, $\delta_i$ at Q1 will be the same as the pctPortfolio. In other words, consider pctPortfolio at the time before it appeared as 0.

2. If investor i invest an instrument j at time T-1, sold all of it at time T , fill in the pctportfolio at the date T as 0. In other words, add a new row for instrument j with pctportfolio as 0 and $\delta_T = 0 - pctPortfolio_{T-1}$ 

3. If investor i invest an instrument j at time T-1, sold all of it at time T and buy it at time T+1, fill in the pctportfolio at the date T as 0.


### instrument

In [ ]:
df_filled = pd.pivot_table(df,index = ['LGCYINVESTORID','INSTRID'], 
                           columns = ['HOLDDATE'],values=['PCTPORTFOLIO_13F']).fillna(0).stack().reset_index()
# get previous portofolio and filled in 0(the first time period will be how much it increased)
df_filled['PREVIOUS_PCTPORTFOLIO'] = df_filled.groupby(['LGCYINVESTORID','INSTRID'])['PCTPORTFOLIO_13F'].shift(1).fillna(0)
# filled in 0 in 'PREVIOUS_PCTPORTFOLIO'   
df_filled = df_filled[(df_filled['PCTPORTFOLIO_13F'] != 0) | (df_filled['PREVIOUS_PCTPORTFOLIO'] != 0)]
df_filled['Delta'] =  df_filled['PCTPORTFOLIO_13F'] - df_filled['PREVIOUS_PCTPORTFOLIO']                    

In [ ]:
df_filled.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/delta.csv')

In [ ]:
# variance for a particular investor and particular time
V_it = df_filled.groupby(['LGCYINVESTORID','HOLDDATE']).agg({'Delta': 'var'})
# V_it.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/variance_it_instrument.csv')

# quantile range for a particular investor and particular time
# V_it = df_filled.groupby(['LGCYINVESTORID','HOLDDATE'])['Delta'].quantile(q=0.75, interpolation='linear') - df_filled.groupby(['LGCYINVESTORID','HOLDDATE'])['Delta'].quantile(q=0.25, interpolation='linear')
# V_it.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/quantile_range_it_instrument.csv')

In [ ]:
# variance for a particular instrument and particular time
# V_jt = df_filled.groupby(['INSTRID','HOLDDATE']).agg({'Delta': 'var'})
# V_jt.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/variance_jt_instrument.csv')

# quantile range for a particular instrument and particular time
V_jt = df_filled.groupby(['INSTRID','HOLDDATE'])['Delta'].quantile(q=0.75, interpolation='linear') - df_filled.groupby(['INSTRID','HOLDDATE'])['Delta'].quantile(q=0.25, interpolation='linear')
# V_jt.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/quantile_range_jt_instrument.csv')

In [ ]:
# variance for a particular time
V_t = df_filled.groupby(['HOLDDATE']).agg({'Delta': 'var'})
# V_t.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/variance_t_instrument.csv')

# quantile range for a particular time
# V_t = df_filled.groupby(['HOLDDATE'])['Delta'].quantile(q=0.75, interpolation='linear') - df_filled.groupby(['HOLDDATE'])['Delta'].quantile(q=0.25, interpolation='linear')
# V_t.to_csv('/content/gdrive/My Drive/capstone/data for trend/quantile_range_t_instrument.csv')

In [ ]:
# Variance of Vit
V_Vit = V_it.reset_index().groupby(['HOLDDATE']).agg({'Delta': 'var'})
# V_Vit.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/variance_of_quantile_range_it_instrument.csv')
# Variance of Vjt
V_Vjt = V_jt.reset_index().groupby(['HOLDDATE']).agg({'Delta': 'var'})
# V_Vjt.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/variance_of_Vjt_instrument.csv')

In [ ]:
# Quantile range of Vit
V_Vit = V_it.reset_index().groupby(['HOLDDATE'])['Delta'].quantile(q=0.75) - V_it.reset_index().groupby(['HOLDDATE'])['Delta'].quantile(q=0.25)
# V_Vit.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/quantile_range_of_quantile_range_it_instrument.csv')
# Quantile range of Vjt
V_Vjt = V_jt.reset_index().groupby(['HOLDDATE'])['Delta'].quantile(q=0.75) - V_jt.reset_index().groupby(['HOLDDATE'])['Delta'].quantile(q=0.25)
# V_Vjt.to_csv('/content/gdrive/My Drive/capstone/data for trend/225 investors/quantile_range_of_Vjt_instrument.csv')

### market cap (not finished)

In [ ]:
mktcap = pd.read_csv('/content/gdrive/My Drive/capstone/full database/MktCap_MktCapSize_20160630_20200630.csv')
mktcap

### industry (not finished)

In [ ]:
df_industry_raw = pd.read_csv('/content/gdrive/My Drive/capstone/full database/portfolio_with_industry.csv')

In [ ]:
df_industry = df_industry_raw.copy()
df_industry = df_industry[['INSTRID', 'HOLDDATE', 'EndDate', 'HierarchicalId', 'Title']]
df_industry = df_industry.drop_duplicates()

In [ ]:
df_industry['HOLDDATE'] = df_industry['HOLDDATE'].str.replace(r' 00:00:00', '')
df_industry['EndDate'] = df_industry['EndDate'].str.replace(r' 00:00:00.000', '')

# df_industry['HOLDDATE'] = pd.to_datetime(df_industry['HOLDDATE'], format='%Y-%m-%d')
# df_industry['EndDate'] = pd.to_datetime(df_industry['EndDate'], format='%Y-%m-%d')

In [ ]:
df_industry['EndDate'].fillna('2020-11-11', inplace=True)

In [ ]:
print(len(set(df_industry.loc[df_industry['EndDate'] == '2020-11-11',:].INSTRID)))
df_industry_latest = df_industry.loc[df_industry['EndDate'] == '2020-11-11',:].dropna()
len(set(df_industry_latest.INSTRID))

10018


8547

In [ ]:
df_industry_latest['HierarchicalId'] = df_industry_latest['HierarchicalId'].astype(str).str[0:4]

In [ ]:
len(set(df_industry_latest.HierarchicalId))

30

In [ ]:
industry_merge = pd.merge(df_sub, df_industry_latest, how='inner', on=['INSTRID', 'HOLDDATE'])[['LGCYINVESTORID','HOLDDATE','PCTPORTFOLIO_13F','HierarchicalId']]

agg_industry_pivot = pd.pivot_table(industry_merge,index = ['LGCYINVESTORID', 'HOLDDATE'], columns = ['HierarchicalId'],values=['PCTPORTFOLIO_13F'],aggfunc=[np.sum],fill_value=0)

ind_investor_set = set(agg_industry_pivot.index.get_level_values('LGCYINVESTORID'))
ind_holddate_set = set(agg_industry_pivot.index.get_level_values('HOLDDATE'))

In [ ]:
len(set(industry_merge['LGCYINVESTORID']))

199

In [ ]:
# fill industry pct of any missing date with 0

industry_id_list = [HierarchicalId for (_,_,HierarchicalId) in list(agg_industry_pivot.columns)]

agg_industry_pivot.columns = industry_id_list
agg_industry_pivot = agg_industry_pivot.reset_index()
for inv in ind_investor_set:
  holddates = set(agg_industry_pivot[agg_industry_pivot['LGCYINVESTORID']==inv]['HOLDDATE'].to_list())
  if len(holddates) < len(ind_holddate_set):
    missing_dates = holddate_set - holddates
    for md in missing_dates:
      agg_industry_pivot = agg_industry_pivot.append([{'LGCYINVESTORID': inv, 'HOLDDATE':md}])

agg_industry_pivot = agg_industry_pivot.fillna(0)

In [ ]:
agg_industry_pivot = agg_industry_pivot.set_index(['LGCYINVESTORID','HOLDDATE']).stack().reset_index()
agg_industry_pivot = agg_industry_pivot.rename(columns={agg_industry_pivot.columns[-2]:'Industry',agg_industry_pivot.columns[-1]:'PCTPORTFOLIO_13F'})

agg_industry_pivot = agg_industry_pivot.pivot_table(index=['LGCYINVESTORID'],columns=['HOLDDATE', 'Industry'],values=['PCTPORTFOLIO_13F'])
agg_industry_pivot.columns =  agg_industry_pivot.columns.droplevel()